In [222]:
import pandas as pd
import numpy as np
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
import math

In [223]:
df=pd.read_csv("laptop_price.csv",encoding='latin-1')
df

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,1,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37kg,1339.69
1,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,898.94
2,3,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,1.86kg,575.00
3,4,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,macOS,1.83kg,2537.45
4,5,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37kg,1803.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1298,1316,Lenovo,Yoga 500-14ISK,2 in 1 Convertible,14.0,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i7 6500U 2.5GHz,4GB,128GB SSD,Intel HD Graphics 520,Windows 10,1.8kg,638.00
1299,1317,Lenovo,Yoga 900-13ISK,2 in 1 Convertible,13.3,IPS Panel Quad HD+ / Touchscreen 3200x1800,Intel Core i7 6500U 2.5GHz,16GB,512GB SSD,Intel HD Graphics 520,Windows 10,1.3kg,1499.00
1300,1318,Lenovo,IdeaPad 100S-14IBR,Notebook,14.0,1366x768,Intel Celeron Dual Core N3050 1.6GHz,2GB,64GB Flash Storage,Intel HD Graphics,Windows 10,1.5kg,229.00
1301,1319,HP,15-AC110nv (i7-6500U/6GB/1TB/Radeon,Notebook,15.6,1366x768,Intel Core i7 6500U 2.5GHz,6GB,1TB HDD,AMD Radeon R5 M330,Windows 10,2.19kg,764.00


In [224]:
lst=['Gaming',
     'Notebook',
     'inches',
    '4k',
    'Touchscreen',
    'AMD',
    'Intel',
    'Clock_Speed',
    'Ram_Size',
    'SSD',
    'Memory_Size',
    'Mac',
    'Windows',
    'Nvidia_Quadro',
    'Nvidia_GTX',
    'AMDG',
    'Price_rs']

In [225]:
lst1=[]
for i in range(1303):
    lst1.append(0)

In [226]:
for i in lst:
    df[i]=lst1
df.dtypes

laptop_ID             int64
Company              object
Product              object
TypeName             object
Inches              float64
ScreenResolution     object
Cpu                  object
Ram                  object
Memory               object
Gpu                  object
OpSys                object
Weight               object
Price_euros         float64
Gaming                int64
Notebook              int64
inches                int64
4k                    int64
Touchscreen           int64
AMD                   int64
Intel                 int64
Clock_Speed           int64
Ram_Size              int64
SSD                   int64
Memory_Size           int64
Mac                   int64
Windows               int64
Nvidia_Quadro         int64
Nvidia_GTX            int64
AMDG                  int64
Price_rs              int64
dtype: object

In [227]:
for i in range(1303):
    typename=df["TypeName"][i]
    if typename == 'Gaming':
        df['Gaming'][i]=1
    elif typename == 'Notebook':
        df["Notebook"][i]=1
        
    inches=df['inches'][i]
    df['inches'][i]=inches
    
    screen=df['ScreenResolution'][i]
    screen=screen.split()
    if '4k' in screen:
        df['4k'][i]=1
    if 'Touchscreen' in screen:
        df['Touchscreen'][i]=1
        
    cpu=df['Cpu'][i]
    cpu=cpu.split()
    if cpu[0]=='Intel':
        df['Intel'][i]=1
    if cpu[0]=='AMD':
        df['AMD'][i]=1
    cs_str=cpu[-1]
    if cs_str[1]=='G':
        cs=float(cs_str[0])
    else:
        cs=float(cs_str[0:3])
    df['Clock_Speed'][i]=cs
    
    ram=df['Ram'][i]
    ram_size=0
    if ram[1]=='G':
        ram_size=ram[0]
    else:
        ram_size=ram[0:2]
    df['Ram_Size'][i]=int(ram_size)
    
    storage=df['Memory'][i]
    if 'SSD' in storage:
        df['SSD'][i]=1
    if storage.split()[0]=='1.0TB' or storage.split()[0]=='1TB' or storage.split()[0]=='2TB':
        df['Memory_Size'][i]=int(storage[0][0])*1024
    else:
        df['Memory_Size'][i]=int(((storage.split()[0]).split('GB')[0]))
        
    os=df['OpSys'][i]
    os=os.split()
    if ('Mac' in os) or ('macOS' in os):
        df['Mac'][i]=1
    if 'Windows' in os:
        df['Windows'][i]=1
        
    gpu=df['Gpu'][i]
    gpu=gpu.split()
    if gpu[0]=='AMD':
        df['AMDG'][i]=0
    if gpu[0]=='Nvidia':
        if gpu[1][0]=='G':
            df['Nvidia_GTX'][i]=1
        else:
            if gpu[1][0]=='Q':
                df["Nvidia_Quadro"][i]=1
  

df['Price_rs']=df['Price_euros']*90
df

C:\Users\shiva\AppData\Local\Temp\ipykernel_2728\3119933385.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['inches'][i]=inches
C:\Users\shiva\AppData\Local\Temp\ipykernel_2728\3119933385.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Intel'][i]=1
C:\Users\shiva\AppData\Local\Temp\ipykernel_2728\3119933385.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Clock_Speed'][i]=cs
C:\Users\shiva\AppData\Local\Temp\ipy

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,...,Clock_Speed,Ram_Size,SSD,Memory_Size,Mac,Windows,Nvidia_Quadro,Nvidia_GTX,AMDG,Price_rs
0,1,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,...,2.3,8,1,128,1,0,0,0,0,120572.1
1,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,...,1.8,8,0,128,1,0,0,0,0,80904.6
2,3,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,...,2.5,8,1,256,0,0,0,0,0,51750.0
3,4,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,...,2.7,16,1,512,1,0,0,0,0,228370.5
4,5,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,...,3.1,8,1,256,1,0,0,0,0,162324.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1298,1316,Lenovo,Yoga 500-14ISK,2 in 1 Convertible,14.0,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i7 6500U 2.5GHz,4GB,128GB SSD,Intel HD Graphics 520,...,2.5,4,1,128,0,1,0,0,0,57420.0
1299,1317,Lenovo,Yoga 900-13ISK,2 in 1 Convertible,13.3,IPS Panel Quad HD+ / Touchscreen 3200x1800,Intel Core i7 6500U 2.5GHz,16GB,512GB SSD,Intel HD Graphics 520,...,2.5,16,1,512,0,1,0,0,0,134910.0
1300,1318,Lenovo,IdeaPad 100S-14IBR,Notebook,14.0,1366x768,Intel Celeron Dual Core N3050 1.6GHz,2GB,64GB Flash Storage,Intel HD Graphics,...,1.6,2,0,64,0,1,0,0,0,20610.0
1301,1319,HP,15-AC110nv (i7-6500U/6GB/1TB/Radeon,Notebook,15.6,1366x768,Intel Core i7 6500U 2.5GHz,6GB,1TB HDD,AMD Radeon R5 M330,...,2.5,6,0,1024,0,1,0,0,0,68760.0


In [228]:
X=np.array(df[['Gaming','Notebook','inches','4k','Touchscreen','AMD','Intel','Clock_Speed','Ram_Size',
    'SSD','Memory_Size','Mac','Windows','Nvidia_Quadro','Nvidia_GTX','AMDG']])
y=np.array(df['Price_rs'])

In [229]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.1)
clf=LinearRegression()
clf.fit(X_train, y_train)
accuracy = clf.score(X_test, y_test)
print(accuracy)

0.7645467889693818


In [237]:
x=[1,0,15,0,0,1,0,2.8,8,0,512,0,1,0,1,0]
for i in range(len(x)):
    if x[i]!=0:
        print(lst[i],x[i])
x=np.array(x)
x=x.reshape(1,-1)
price=int(clf.predict(x))
mar=price*(1-accuracy)
print(price,'+-',int(mar),sep='')

Gaming 1
inches 15
AMD 1
Clock_Speed 2.8
Ram_Size 8
Memory_Size 512
Windows 1
Nvidia_GTX 1
70169+-16521
